In [1]:
import numpy as np
import random

In [8]:
def generate_random_demand(size):
    return [random.choice([0, 1, 2, 3]) for _ in range(size)]

num_simulations = 5000

holding_cost = 5
transshipment_cost = 30
profit_lost_cost = 150

max_inventory = 20

In [9]:
def simulate():
    demand1 = generate_random_demand(7)
    demand2 = generate_random_demand(7)
    
    initial_inventory1 = 12
    initial_inventory2 = 12

    T = len(demand1)

    dp = np.full((T + 1, max_inventory + 1, max_inventory + 1), np.inf)
    dp[0, initial_inventory1, initial_inventory2] = 0

    transfer_records = np.zeros((T, max_inventory + 1, max_inventory + 1), dtype=int)

    for t in range(T):
        for i in range(max_inventory + 1):
            for j in range(max_inventory + 1):
                if dp[t, i, j] < np.inf:
                    for transfer in range(-min(i, 12), min(j, 12) + 1):
                        inventory1 = i - demand1[t] + transfer
                        inventory2 = j - demand2[t] - transfer
                        if 0 <= inventory1 <= max_inventory and 0 <= inventory2 <= max_inventory:
                            holding_costs = (inventory1 + inventory2) * holding_cost
                            transshipment_costs = abs(transfer) * transshipment_cost
                            lost_sales_costs = (max(0, demand1[t] - i) + max(0, demand2[t] - j)) * profit_lost_cost
                            total_cost = holding_costs + transshipment_costs + lost_sales_costs
                            if dp[t + 1, inventory1, inventory2] > dp[t, i, j] + total_cost:
                                dp[t + 1, inventory1, inventory2] = dp[t, i, j] + total_cost
                                transfer_records[t, inventory1, inventory2] = transfer

    optimal_cost = np.min(dp[T])
    optimal_inventory1, optimal_inventory2 = np.unravel_index(np.argmin(dp[T]), (max_inventory + 1, max_inventory + 1))

    transfers = []
    for t in range(T - 1, -1, -1):
        transfer = transfer_records[t, optimal_inventory1, optimal_inventory2]
        transfers.append(transfer)
        optimal_inventory1 += demand1[t] - transfer
        optimal_inventory2 += demand2[t] + transfer
    transfers.reverse()

    return optimal_cost, transfers

In [10]:
all_costs = []
all_transfers = []

total_transfers_1_to_2 = 0
total_transfers_2_to_1 = 0

for _ in range(num_simulations):
    cost, transfers = simulate()
    if cost < np.inf:
        all_costs.append(cost)
        all_transfers.append(transfers)
        for transfer in transfers:
            if transfer > 0:
                total_transfers_1_to_2 += transfer
            elif transfer < 0:
                total_transfers_2_to_1 += -transfer

In [11]:
average_cost = np.mean(all_costs)
std_cost = np.std(all_costs)

In [12]:
print(f"2000 simülasyonun ortalama maliyeti: {average_cost}")
print(f"2000 simülasyonun toplam maliyetlerinin standart sapması: {std_cost}")
print(f"Store 1'den Store 2'ye yapılan toplam transfer sayısı: {total_transfers_1_to_2}")
print(f"Store 2'den Store 1'e yapılan toplam transfer sayısı: {total_transfers_2_to_1}")

2000 simülasyonun ortalama maliyeti: 465.12833545108003
2000 simülasyonun toplam maliyetlerinin standart sapması: 71.31245596853039
Store 1'den Store 2'ye yapılan toplam transfer sayısı: 990
Store 2'den Store 1'e yapılan toplam transfer sayısı: 1181


In [13]:
for i, (cost, transfers) in enumerate(zip(all_costs, all_transfers), start=1):
    print(f"Simülasyon {i}: Toplam maliyet = {cost}")
    for t, transfer in enumerate(transfers, start=1):
        if transfer > 0:
            print(f"Gün {t}: Mağaza 1'den Mağaza 2'ye {transfer} adet transfer")
        elif transfer < 0:
            print(f"Gün {t}: Mağaza 2'den Mağaza 1'e {-transfer} adet transfer")
        else:
            print(f"Gün {t}: Transfer yok")


Simülasyon 1: Toplam maliyet = 570.0
Gün 1: Transfer yok
Gün 2: Transfer yok
Gün 3: Transfer yok
Gün 4: Transfer yok
Gün 5: Transfer yok
Gün 6: Transfer yok
Gün 7: Transfer yok
Simülasyon 2: Toplam maliyet = 445.0
Gün 1: Transfer yok
Gün 2: Transfer yok
Gün 3: Transfer yok
Gün 4: Transfer yok
Gün 5: Transfer yok
Gün 6: Transfer yok
Gün 7: Transfer yok
Simülasyon 3: Toplam maliyet = 390.0
Gün 1: Transfer yok
Gün 2: Transfer yok
Gün 3: Transfer yok
Gün 4: Transfer yok
Gün 5: Transfer yok
Gün 6: Transfer yok
Gün 7: Transfer yok
Simülasyon 4: Toplam maliyet = 495.0
Gün 1: Transfer yok
Gün 2: Transfer yok
Gün 3: Transfer yok
Gün 4: Transfer yok
Gün 5: Transfer yok
Gün 6: Mağaza 1'den Mağaza 2'ye 2 adet transfer
Gün 7: Transfer yok
Simülasyon 5: Toplam maliyet = 365.0
Gün 1: Mağaza 2'den Mağaza 1'e 1 adet transfer
Gün 2: Transfer yok
Gün 3: Transfer yok
Gün 4: Transfer yok
Gün 5: Transfer yok
Gün 6: Transfer yok
Gün 7: Transfer yok
Simülasyon 6: Toplam maliyet = 415.0
Gün 1: Transfer yok
Gün